## Сравнение некоторых отобранных методов / моделей для векторизации

Способы:
- user/bge-m3
- mlsa-iai-msu-lab/sci-rus-tiny
- tf-ifd

Метрика:
- косинусная близость 

В качесвте образца для тестов выберем один отчет, который предобработаем уюрав лишние символы, упоминание таблиц и изображений (их по предыдущим экспериментам необходимо обрабатывать отдельно иначе процент попадания в нужный текст очень мал) 

In [42]:
# импорт библиотек
import numpy as np
import pandas as pd
import json
import uuid

from langchain_core.documents import Document
from sentence_transformers import SentenceTransformer

import torch
import torch.nn.functional as F 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel

from typing import List

### Вариант 1. Обработанных отчет, с сохранением цифр вне таблиц

In [5]:
# загрузка заранее разбитых документов
input_file_path = "../../data/processed_json_all/all_documents_512.json"

# Чтение данных из JSON
with open(input_file_path, "r", encoding="utf-8") as json_file:
    data = json.load(json_file)

# Восстановление массива объектов Document
documents = [
    Document(metadata=item["metadata"], page_content=item["page_content"])
    for item in data
]

# возьмем для оценка бассейн Дон 
print(f"Загружено {len(documents)} фрагментов текста.")

filtered_documents = [
    doc for doc in documents
    if doc.metadata.get('basin') == 'Дон' and 'Книга_6.json' in doc.metadata.get('file', '') and doc.page_content!=''
]
print(f"Отфильтрованно {len(filtered_documents)} фрагментов текста.")
print(filtered_documents[:5])

Загружено 3597 фрагментов текста.
Отфильтрованно 401 фрагментов текста.
[Document(metadata={'title': 'ВВедение', 'level': 1, 'parent_title': '', 'start_page': 11, 'end_page': 12, 'file': 'Книга_6.json', 'basin': 'Дон', 'chunk_index': 1, 'total_chunks': 2, 'order': 1567}, page_content='Введение В Книге 6 приводится перечень мероприятий, направленных на сохранение и восстанов ление водных объектов в бассейне р. Дон и обеспечивающих устойчивое функционирование водохозяйственных систем в рамках бассейна на основе достижения установленных целевых показателей. Перспективы развития водохозяйственного комплекса бассейна и формирование си стемы мероприятий осуществлялось в соответствии с ключевыми положениями Водной стратегии Российской Федерации на период до 2020 года. При формировании вариантов программ мероприятий, разрабатываемых в проекте Схемы, учитывались стратегии долгосрочного социально экономического развития Российской Федерации и регионов, федеральные, региональные и отраслевые целе

In [8]:
# Извлечение page_content из документов
texts = [doc.page_content for doc in filtered_documents]

# 1. USER-bge-m3 embeddings
model_bge_m3 = SentenceTransformer("deepvk/USER-bge-m3") 
embeddings_bge_m3 = model_bge_m3.encode(texts)

# 2. TF-IDF vectors
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectors = tfidf_vectorizer.fit_transform(texts).toarray()

# 3. Sci-Rus-Tiny embeddings
model_sci_rus_tiny = SentenceTransformer("mlsa-iai-msu-lab/sci-rus-tiny")
embeddings_sci_rus_tiny = model_sci_rus_tiny.encode(texts)

In [33]:
# Сохранение результатов в DataFrame
df_results = pd.DataFrame({
    "page_comtent": texts,
    "Metadata": [doc.metadata for doc in filtered_documents],
    "USER-bge-m3": list(embeddings_bge_m3),
    "TF-IDF": list(tfidf_vectors),
    "Sci-Rus-Tiny": list(embeddings_sci_rus_tiny),
})

# Сохранение в файл для дальнейшего анализа
df_results.to_pickle("document_embeddings.pkl")

# Вывод результатов для проверки
df_results.head(5)

,page_comtent,Metadata,USER-bge-m3,TF-IDF,Sci-Rus-Tiny
0,Введение В Книге 6 приводится перечень меропри...,"{'title': 'ВВедение', 'level': 1, 'parent_titl...","[0.046000365, 0.0030327977, -0.020319164, -0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.024866175, -0.04918961, 0.0058632516, -0.0..."
1,ственного комплекса и регулирования водопользо...,"{'title': 'ВВедение', 'level': 1, 'parent_titl...","[0.039340682, 0.015985286, -0.044753596, -0.01...","[0.02315751628302318, 0.0, 0.0, 0.0, 0.0, 0.0,...","[-0.033156816, -0.0031238478, 0.029817969, 0.0..."
2,1 Цели И Задачи Программы Мероприятий По Дости...,{'title': '1 цели и задачи программы мероприят...,"[0.018332295, -0.014730563, -0.041701123, -0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.045997083, -0.022196649, -0.050526712, 0.0..."
3,воды в водопроводящих элементах водохозяйствен...,{'title': '1 цели и задачи программы мероприят...,"[0.022883212, 0.018903986, -0.028815147, -0.00...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.06186741, -0.07329733, 0.026311874, 0.0166..."
4,"ие водных объектов до состояния, обеспечивающе...",{'title': '1 цели и задачи программы мероприят...,"[0.015333965, 0.014754793, -0.0228113, -0.0146...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.07004554, 0.008788516, 0.008739165, -0.007..."


In [32]:
# Введите запрос
query = "Расскажи про Строительство и реконструкция очистных сооружений"
query_embedding_bge = model_bge_m3.encode([query])[0]
query_tfidf_vectors = tfidf_vectorizer.fit_transform(query).toarray()
embeddings_sci_rus_tiny = model_sci_rus_tiny.encode(query)

# Косинусное сходство между запросом и эмбеддингами документов
cosine_similarities = cosine_similarity([query_embedding_bge], embeddings_bge_m3).flatten()

# Сортировка документов по сходству
sorted_indices = cosine_similarities.argsort()[::-1]

# Вывод топ-5 результатов
print("Top-5 most similar documents:")
for idx in sorted_indices[:5]:
    print(f"Document: {texts[idx]}")
    print(f"Similarity: {cosine_similarities[idx]}\n")

Top-5 most similar documents:
Document: 1.84 0.67 Строительство и реконструкция очистных сооружений Разработка проектно смет ной документации строи тельства локальных очист ных сооружений в г. Ат карске 0.50 0.50 0.50 Мб 0.50 05.01.03.001 р. Аткара (приток р. Медведица) Строительство локальных очистных сооружений в г. Аткарске 5.00 5.00 5.00 Фб, Об 4.50 0.50 05.01.03.001 р. Аткара (приток р. Медведица) Разработка проектно смет ной документации ре конструкции канализаци онных очистных сооруже ний в г. Балашове 2.50 2.50 2.50 Мб 2.50 05.01.02.001 р. Хопер Реконструкция городских очистных сооружений канализации, г. Балашов 67.00 67.00 33.50 33.50 Фб, Об 60.30 6.70 05.01.02.001 р. Хопер Разработка проектно смет ной документации ре конструкции канализаци онных очистных сооруже ний в г. Калининске 1.90 1.90 1.90 Мб 1.90 05.01.03.001 р. Баланда (приток р. Медведица) Реконструкция городских очистных сооружений канализации, г. Калининск 21.50 21.50 21.50 Фб, Об 19.35 2.15 05.01.03.001 р. Баланд

In [41]:
# Функция для поиска топ-5 результатов
def find_top_results(query_vector, doc_vectors, texts, metadata, metric="cosine"):
    similarities = []
    for i, doc_vector in enumerate(doc_vectors):
        if metric == "cosine":
            # Косинусная близость
            similarity = cosine_similarity([query_vector], [doc_vector])[0][0]
        similarities.append((i, similarity, texts[i], metadata[i]))

    # Сортировка по убыванию сходства
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:5]  # Возвращаем топ-5 результатов

# Введите запрос
query = "За счет чего снижается негативное воздействия вод?"

# Векторизация запроса
query_embedding_bge = model_bge_m3.encode([query])[0]
query_tfidf_vectors = tfidf_vectorizer.transform([query]).toarray()[0]
query_embedding_sci_rus_tiny = model_sci_rus_tiny.encode([query])[0]

# Поиск топ-5 результатов для каждой модели
top_bge_cosine = find_top_results(query_embedding_bge, embeddings_bge_m3, texts, [doc.metadata for doc in filtered_documents], metric="cosine")
top_tfidf_cosine = find_top_results(query_tfidf_vectors, tfidf_vectors, texts, [doc.metadata for doc in filtered_documents], metric="cosine")
top_sci_rus_cosine = find_top_results(query_embedding_sci_rus_tiny, embeddings_sci_rus_tiny, texts, [doc.metadata for doc in filtered_documents], metric="cosine")

# Функция для вывода результатов
def print_results(results, model_name, metric):
    print(f"Топ-5 результатов для модели {model_name} (метрика: {metric}):")
    for i, (idx, similarity, text, metadata) in enumerate(results):
        print(f"{i + 1}. Сходство: {similarity:.4f}")
        print(f"   Текст: {text}...") 
        print(f"   Метаданные: {metadata}")
        print("-" * 80)

# Вывод результатов
print_results(top_bge_cosine, "USER-bge-m3", "косинусная близость")
print_results(top_tfidf_cosine, "TF-IDF", "косинусная близость")
print_results(top_sci_rus_cosine, "Sci-Rus-Tiny", "косинусная близость")

Топ-5 результатов для модели USER-bge-m3 (метрика: косинусная близость):
1. Сходство: 0.6997
   Текст: 4.2 Снижение негативного воздействия вод на население и объекты экономики Снижение негативного воздействия вод в бассейне обеспечивается за счет  проведения противопаводковых расчисток русел рек  строительства, реконструкции и капитального ре  монта гидротехнических сооружений  строительства берегоукрепительных сооружений. Поэтапное выполнение этих мероприятий позволяет, в конечном итоге, уменьшить количе  ственные значения показателей негативного воздействия вод до уровня целевого состояния. В качестве основных показателей снижения негативного воздействия вод рассмат  риваются  уменьшение площади освоенной территории, подверженной негативному воздей  ствию  уменьшение количества населения, проживающего на периодически затапливаемых тер  риториях  уменьшение величины ущерба, возникающего в результате негативного воздействия вод  сокращение количества гидротехнических сооружений, наход

Модель USER-bge-m3 сразу увидела и хорошо оценила ответ, при этом дала высокую оценку, что ценно при дальнейшем ранжировании ответов.
TF-IDF - нашел ответ, но не дал хорошой оценки по косинусной близости.
Модель Sci-Rus-Tiny  - вообще промазала